# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [87]:
# import libraries
import pandas as pd
import re

from nltk.tokenize import word_tokenize
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords 
from nltk.stem import WordNetLemmatizer
from sklearn.pipeline import Pipeline 
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
import nltk
import pickle
nltk.download(['punkt','stopwords','wordnet'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [28]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('InsertTableName', con = engine.connect())
feature_cols = list(df.columns)[4:]
X = df['message']  # Message Column
y = df[feature_cols] # Classification label

In [29]:
y.head()

,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,water,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [30]:
def tokenize(text):
    """
    Split text into words and return the root form of the words
    
    Args:
      text (str): the message
      
    Return:
      lemm (list of str): a list of the root form of the message words
    
    """
    
    # Normalize text
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # Tokenize text
    words = word_tokenize(text)
    
    # Remove stop words
    stop = stopwords.words("english")
    words = [t for t in words if t not in stop]
    
    # Lemmatization
    lemm = [WordNetLemmatizer().lemmatize(w) for w in words]
    
    return lemm

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [71]:
# Pipleine: Random Forest Classifier

pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer = tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf',  RandomForestClassifier(random_state = 42))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [72]:
# Splitting data
X_train, X_test, y_train, y_test = train_test_split(X, y)
                                                    
# Fit the Random Forest Classifier 
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...stimators=10, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [33]:
# Prediction: Random Forest Classifier  
y_pred = pipeline.predict(X_test)

In [44]:
# Plot classification reports(f1-score, precision, recall)  for each feature
for idx, col in enumerate(y_test):
    print('Feature: {}'.format(col))
    print(classification_report(y_test[col], y_pred[:, idx]))
# compute and plot model accuracy   
accuracy = (y_test.values == y_pred).mean()       
print('Model accuracy: {}'.format(accuracy))    

Feature: request
             precision    recall  f1-score   support

          0       0.90      0.98      0.94      5458
          1       0.80      0.45      0.57      1096

avg / total       0.88      0.89      0.88      6554

Feature: offer
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      6520
          1       0.00      0.00      0.00        34

avg / total       0.99      0.99      0.99      6554

Feature: aid_related
             precision    recall  f1-score   support

          0       0.71      0.90      0.79      3871
          1       0.77      0.47      0.58      2683

avg / total       0.73      0.72      0.71      6554

Feature: medical_help
             precision    recall  f1-score   support

          0       0.93      1.00      0.96      6060
          1       0.57      0.02      0.05       494

avg / total       0.90      0.93      0.89      6554

Feature: medical_products
             precision    recall  f1-sc

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [45]:
# Show parameters for the pipline
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7fdebfab97b8>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=42, verbose=0, warm_start=False))],
 

In [50]:
# Create Grid search parameters for Random Forest Classifier   
parameters = {
        'tfidf__use_idf': (True, False),
        'clf__n_estimators': [1, 10, 20]
}

gridsearch = GridSearchCV(pipeline, param_grid = parameters)
gridsearch

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...stimators=10, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'tfidf__use_idf': (True, False), 'clf__n_estimators': [1, 10, 20]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [51]:
# Fit the Random Forest Classifier using GridSearch
cv.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...stimators=10, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'tfidf__use_idf': (True, False), 'clf__n_estimators': [1, 10, 20]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [52]:
# Prediction with GridSearch: Random Forest Classifier  
y_pred = cv.predict(X_test)

In [53]:
# Plot classification reports(f1-score, precision, recall)  for each feature
for idx, col in enumerate(y_test):
    print('Feature: {}'.format(col))
    print(classification_report(y_test[col], y_pred[:, idx]))
# compute and plot model accuracy   
accuracy = (y_test.values == y_pred).mean()       
print('Model accuracy: {}'.format(accuracy))

Feature: request
             precision    recall  f1-score   support

          0       0.90      0.98      0.94      5458
          1       0.81      0.48      0.60      1096

avg / total       0.89      0.89      0.88      6554

Feature: offer
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      6520
          1       0.00      0.00      0.00        34

avg / total       0.99      0.99      0.99      6554

Feature: aid_related
             precision    recall  f1-score   support

          0       0.73      0.90      0.81      3871
          1       0.78      0.52      0.62      2683

avg / total       0.75      0.74      0.73      6554

Feature: medical_help
             precision    recall  f1-score   support

          0       0.93      1.00      0.96      6060
          1       0.58      0.02      0.04       494

avg / total       0.90      0.93      0.89      6554

Feature: medical_products
             precision    recall  f1-sc

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

#### 8.1 Bagging Ensemble Classifier

In [73]:
# Create pipeline with Adaboost Classifier
pipeline_ada = Pipeline([
        ('vect', CountVectorizer(tokenizer = tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf',  MultiOutputClassifier(AdaBoostClassifier()))
    ])

In [77]:
# Fit the model
pipeline_ada.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))])

In [78]:
# Prediction with Adaboost Classifier  
y_pred = pipeline_ada.predict(X_test)

In [79]:
# Plot classification reports(f1-score, precision, recall)  for each feature
for idx, col in enumerate(y_test):
    print('Feature: {}'.format(col))
    print(classification_report(y_test[col], y_pred[:, idx]))
# compute and plot model accuracy   
accuracy = (y_test.values == y_pred).mean()       
print('Model accuracy: {}'.format(accuracy))  

Feature: request
             precision    recall  f1-score   support

          0       0.91      0.97      0.93      5422
          1       0.76      0.52      0.62      1132

avg / total       0.88      0.89      0.88      6554

Feature: offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6527
          1       0.09      0.04      0.05        27

avg / total       0.99      0.99      0.99      6554

Feature: aid_related
             precision    recall  f1-score   support

          0       0.75      0.86      0.80      3785
          1       0.76      0.61      0.68      2769

avg / total       0.76      0.76      0.75      6554

Feature: medical_help
             precision    recall  f1-score   support

          0       0.94      0.98      0.96      6035
          1       0.58      0.27      0.37       519

avg / total       0.91      0.93      0.91      6554

Feature: medical_products
             precision    recall  f1-sc

In [80]:
# Show parameters for the pipline
pipeline_ada.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7fdebfab97b8>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
             learning_rate=1.0, n_estimators=50, random_state=None),
              n_jobs=1))],
 'vect': CountVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
         lowercase=True, max_df=1.0, max_features=None, min_df=1,
         ngram_rang

In [82]:
# Create Grid search parameters for Adaboost Classifier   
parameters_ada = {
        'tfidf__use_idf': (True, False),
        'clf__estimator__n_estimators': [10,20,50],
}

gridsearch_ada = GridSearchCV(pipeline_ada, param_grid = parameters_ada)
gridsearch_ada

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'tfidf__use_idf': (True, False), 'clf__estimator__n_estimators': [10, 20, 50]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [83]:
# Fit GridSearchCV for Adaboost model
gridsearch_ada.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'tfidf__use_idf': (True, False), 'clf__estimator__n_estimators': [10, 20, 50]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [84]:
# Prediction with Adaboost Classifier  
y_pred = gridsearch_ada.predict(X_test)

In [85]:
# Plot classification reports(f1-score, precision, recall)  for each feature
for idx, col in enumerate(y_test):
    print('Feature: {}'.format(col))
    print(classification_report(y_test[col], y_pred[:, idx]))
# compute and plot model accuracy   
accuracy = (y_test.values == y_pred).mean()       
print('Model accuracy: {}'.format(accuracy))  

Feature: request
             precision    recall  f1-score   support

          0       0.90      0.96      0.93      5422
          1       0.73      0.51      0.60      1132

avg / total       0.87      0.88      0.87      6554

Feature: offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6527
          1       0.00      0.00      0.00        27

avg / total       0.99      0.99      0.99      6554

Feature: aid_related
             precision    recall  f1-score   support

          0       0.73      0.87      0.79      3785
          1       0.75      0.56      0.65      2769

avg / total       0.74      0.74      0.73      6554

Feature: medical_help
             precision    recall  f1-score   support

          0       0.94      0.99      0.96      6035
          1       0.58      0.21      0.30       519

avg / total       0.91      0.93      0.91      6554

Feature: medical_products
             precision    recall  f1-sc

### 9. Export your model as a pickle file

In [89]:
# Export model as pickle file
file_name = 'classifier.pkl'
with open (file_name, 'wb') as file:
    pickle.dump(gridsearch_ada, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.